In [3]:
import pandas as pd
import numpy as np
import requests
from tqdm import tqdm
import datetime
from io import BytesIO

In [4]:
import os
os.getcwd()

'/data/hun/Thesis'

In [5]:
krx = '/data/hun/KRX_marketdata/'

In [8]:
fin1 = pd.read_csv(krx+'kospi_fin.csv')

In [19]:
tickers = pd.read_csv(krx+'ticker_recent.csv', encoding ='cp949')
tickers.columns = ('full_ticker', 'ticker', 'name_full', 'name',
              'name_eng', 'listed_data', 'market', 'security_category',
              'related_department', 'preferred', 'face_value', 'shares')
tickers = tickers.drop(['related_department'], axis=1).dropna().reset_index(drop=True)

In [24]:
tickers[tickers.name =='인바이오젠']

,full_ticker,ticker,name_full,name,name_eng,listed_data,market,security_category,preferred,face_value,shares
1755,KR7101140002,101140,인바이오젠보통주,인바이오젠,INBIOGEN,2008/04/02,KOSPI,주권,보통주,500,51561397


In [9]:
def KRX_price_otp(full_ticker, ticker, name):
    url = "http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd"
    headers = {"Origin": "http://data.krx.co.kr",
               "Referer": "http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020201",
               "User-Agent": "Safari/537.36"}

#     data = {
#         "tboxisuCd_finder_stkisu0_3": f"{ticker}/{name}",
#         "searchType": '2',
#         "mktld":"ALL",
#         "trdDd": '20211112',
#         "isuCd": f"{full_ticker}",
#         "isuCd2": f"{full_ticker}",
#         "param1isuCd_finder_stkisu0_3": "ALL",
#         "strtDd": '20000101',  
#         "endDd": '20210901', 
#         "csvxls_isNo": "false",
#         "name": "fileDown",
#         "url": "dbms/MDC/STAT/standard/MDCSTAT03502" #daily주의 바꿔야하는부분
#     }
    
    data = {
        "searchType": "2",
        "mktId": "ALL",
        "trdDd": "20211112",
#         "tboxisuCd_finder_stkisu0_3": "098120/마이크로컨텍솔",
        "isuCd": f"{full_ticker}",
        "isuCd2": f"{full_ticker}",
#         "codeNmisuCd_finder_stkisu0_3": f"{name}",
        "param1isuCd_finder_stkisu0_3": "ALL",
        "strtDd": "19970101",
        "endDd": "20210901",
        "csvxls_isNo": "false",
        "name": "fileDown",
        "url": "dbms/MDC/STAT/standard/MDCSTAT03502"
    }

    resp = requests.post(url, headers=headers, data=data)

    # if resp.json()['title'] == 'Access Denied':
    # raise Exception("otp problem, access denied")
    # else:
    try:
        return resp.text
    except:
        print(name, ticker, resp)


def KRX_price_get(full_ticker, ticker, name):

    otp = KRX_price_otp(full_ticker, ticker, name)
    url = "http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd"
    header = {"Connection": "keep-alive",
              "Host": "data.krx.co.kr",
              "Origin": "http://data.krx.co.kr",
              "User-Agent": "Mozilla/5.0",
              "Referer": "http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020201"} #daily부분 변경
    data = {'code': otp}
    resp = requests.post(url, headers=header, data=data)
    if resp.status_code == 200:
        df = pd.read_csv(BytesIO(resp.content), encoding='cp949')
        df['ticker'] = ticker
        df['name'] = name
        df = rename_KRX(df)
        return df


def rename_KRX(data):
    data = data.rename(columns={'일자': 'date', '시가': 'open', '고가': 'high',
                                '저가': 'low', '종가': 'close', '거래량': 'volume',
                                '대비': "daily_change", '등락률': 'daily_return', '거래대금': 'transaction_amount',
                                '시가총액': 'mktcap', '상장주식수': "stock_avail", "최고가(종가)": 'high',
                                "최저가(종가)": "low", "거래량_합계": "total_volume", 
                                "거래대금_합계": "total_transactions", "거래량_일평균":"avg_volume",
                                "거래대금_일평균": "avg_transactions", "연월": "date"})
    cols = data.columns.to_list()
    cols = [cols[0]] + cols[-2:] + cols[1:-2]
    return data.reindex(columns=cols)



In [22]:
table = 'market_krx_daily'
# cols = 'date,ticker, name, open, high, low, close, volume, \
#     daily_change, daily_return, transaction_amount, mktcap, stock_avail'
error = []
error_name = []
cnt=0
for i in tqdm(tickers.values):
    full_ticker, ticker, name = i[0], i[1], i[3]
    try:
        res = KRX_price_get(full_ticker, ticker, name)
        res['date'] = res.date.apply(lambda x: datetime.datetime.strptime(x,"%Y/%m/%d"))
        res['date2'] = res.date.apply(lambda x: (x.year, x.month))
        res.drop_duplicates(subset = ['date2'], keep="last", inplace = True)
        
        if cnt == 0:
            res.to_csv(krx+'krx_fin_monthly2.csv', index=False, header = True)
            cnt = 1
        else:
            res.to_csv(krx+'krx_fin_monthly2.csv', index=False, header =False, mode='a')
    except Exception as e:
        print(e)
        print(ticker, name)
        error.append(ticker)
        error_name.append(name)
pd.Series(error, index = error_name).to_csv(krx+"krx_fin_m2_error.csv")


100%|██████████| 2600/2600 [2:27:13<00:00,  3.40s/it]  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [13]:
res

,date,ticker,name,close,daily_change,daily_return,EPS,PER,BPS,PBR,주당배당금,배당수익률,date2
0,2021-09-01,015750,성우하이텍,6210,-10,-0.16,0,0.0,12845,0.48,50.0,0.81,"(2021, 9)"
21,2021-08-02,015750,성우하이텍,6740,170,2.59,0,0.0,12845,0.52,50.0,0.74,"(2021, 8)"
43,2021-07-01,015750,성우하이텍,7440,120,1.64,0,0.0,12845,0.58,50.0,0.67,"(2021, 7)"
65,2021-06-01,015750,성우하이텍,7220,310,4.49,0,0.0,12845,0.56,50.0,0.69,"(2021, 6)"
84,2021-05-03,015750,성우하이텍,5870,230,4.08,0,0.0,12845,0.46,50.0,0.85,"(2021, 5)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6086,1997-05-02,015750,성우하이텍,30100,-2600,-7.95,0,0.0,0,0.00,NaN,NaN,"(1997, 5)"
6111,1997-04-01,015750,성우하이텍,29800,-2500,-7.74,0,0.0,0,0.00,NaN,NaN,"(1997, 4)"
6136,1997-03-03,015750,성우하이텍,18400,0,0.00,0,0.0,0,0.00,NaN,NaN,"(1997, 3)"
6158,1997-02-01,015750,성우하이텍,20000,0,0.00,0,0.0,0,0.00,NaN,NaN,"(1997, 2)"


In [23]:
test = pd.read_csv(krx+"krx_fin_monthly2.csv")
test

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,date,ticker,name,close,daily_change,daily_return,EPS,PER,BPS,PBR,주당배당금,배당수익률,date2
0,2021-09-01,098120,마이크로컨텍솔,8830,-280,-3.07,135,65.41,4134,2.14,25.0,0.28,"(2021, 9)"
1,2021-08-02,098120,마이크로컨텍솔,8390,330,4.09,135,62.15,4134,2.03,25.0,0.30,"(2021, 8)"
2,2021-07-01,098120,마이크로컨텍솔,8140,350,4.49,135,60.30,4134,1.97,25.0,0.31,"(2021, 7)"
3,2021-06-01,098120,마이크로컨텍솔,6470,30,0.47,135,47.93,4134,1.57,25.0,0.39,"(2021, 6)"
4,2021-05-03,098120,마이크로컨텍솔,5840,-20,-0.34,135,43.26,4134,1.41,25.0,0.43,"(2021, 5)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
422041,2017-11-01,238490,힘스,23450,1800,8.31,1541,15.22,5381,4.36,0.0,0.00,"(2017, 11)"
422042,2017-10-10,238490,힘스,21600,400,1.89,1541,14.02,5381,4.01,0.0,0.00,"(2017, 10)"
422043,2017-09-01,238490,힘스,25450,-400,-1.55,1541,16.52,5381,4.73,0.0,0.00,"(2017, 9)"
422044,2017-08-01,238490,힘스,28900,-1750,-5.71,1541,18.75,5381,5.37,0.0,0.00,"(2017, 8)"


In [17]:
test[test.name =='삼양옵틱스']

,date,ticker,name,close,daily_change,daily_return,EPS,PER,BPS,PBR,주당배당금,배당수익률,date2
